In [1]:
%pwd

'/home/jovyan/work/obstacle-detection/examples'

In [2]:
%cd ..

/home/jovyan/work/obstacle-detection


In [3]:
import sys
sys.path.append("..")
from model.tasks.semantic.infer2 import get_user 
#user = get_user("dataset", "darknet21")
user = get_user("dataset", "squeezeseg")

----------
INTERFACE:
dataset dataset
model squeezeseg
----------

Opening arch config file from squeezeseg
Opening data config file from squeezeseg
model folder exists! Using model from squeezeseg
parser started
Using SqueezeNet Backbone
Depth of backbone input =  5
Original OS:  16
New OS:  16
Strides:  [2, 2, 2, 2]
Decoder original OS:  16
Decoder new OS:  16
Decoder strides:  [2, 2, 2, 2]
Total number of parameters:  915540
Total number of parameters requires_grad:  915540
Param encoder  724032
Param decoder  179968
Param head  11540
Successfully loaded model backbone weights
Successfully loaded model decoder weights
Successfully loaded model head weights
Infering in device:  cpu


In [9]:
%timeit predicted_labels = user.infer('dataset/sequences/06/velodyne/000001.bin')

proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
Infered scan  dataset/sequences/06/velodyne/000001.bin in 0.6181917190551758 sec
proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
Infered scan  dataset/sequences/06/velodyne/000001.bin in 0.5907962322235107 sec
proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
Infered scan  dataset/sequences/06/velodyne/000001.bin in 0.6699767112731934 sec
proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
Infered scan  dataset/sequences/06/velodyne/000001.bin in 0.5981595516204834 sec
proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
I

In [5]:
predicted_labels = user.infer('dataset/sequences/06/velodyne/000003.bin')

proj_x shape =  torch.Size([1, 150000])
proj_y shape =  torch.Size([1, 150000])
torch.Size([1, 5, 64, 2048])
torch.Size([1, 64, 2048])
Infered scan  dataset/sequences/06/velodyne/000003.bin in 0.6274764537811279 sec


In [6]:
predicted_labels.shape

(123083,)

In [7]:
predicted_labels

array([13, 16, 19, ...,  9,  9,  9])